In [1]:
import numpy
import pandas as pd
import os
import csv
import ast
import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt

from wquantiles import quantile
from openfisca_survey_manager.utils import asof

from openfisca_france_indirect_taxation import FranceIndirectTaxationTaxBenefitSystem
from openfisca_france_indirect_taxation.examples.utils_example import (
    wavg,
    collapse,
    dataframe_by_group,
    graph_builder_bar,
    df_weighted_average_grouped)
from openfisca_france_indirect_taxation.build_survey_data.utils import weighted_sum
from openfisca_france_indirect_taxation.almost_ideal_demand_system.utils import add_niveau_vie_decile
from openfisca_france_indirect_taxation.surveys import SurveyScenario
from openfisca_france_indirect_taxation.calibration import get_inflators_by_year_energy
from openfisca_france_indirect_taxation.utils import assets_directory, get_input_data_frame

In [2]:
from openfisca_france_indirect_taxation.projects.TVA_Herve_IPP.new_calage_bdf_cn import (
    new_get_bdf_aggregates,
    new_get_cn_aggregates,
    new_get_inflators_bdf_to_cn,
    new_get_inflators_cn_to_cn,
    new_get_inflators,
    new_get_inflators_by_year)

In [29]:
data_year = 2017
inflators_by_year = new_get_inflators_by_year(rebuild = True, year_range = range(2017, 2025), data_year = data_year)
simulated_variables = ['tva_taux_plein',
'tva_taux_intermediaire',
 'tva_taux_reduit',
 'tva_taux_super_reduit',
 'tva_total',
 'depenses_tva_exonere',
 'depenses_tva_taux_plein',
 'depenses_tva_taux_intermediaire',
 'depenses_tva_taux_reduit',
 'depenses_tva_taux_super_reduit',
 'rev_disponible',
 'rev_disp_yc_loyerimpute',
 'loyer_impute',
 'depenses_tot',
 'depenses_totales',
 'pondmen',
]

agregates = pd.DataFrame(columns = simulated_variables)
for year in range(2017,2025) :
    inflation_kwargs = dict(inflator_by_variable = inflators_by_year[year])
    survey_scenario = SurveyScenario.create(
        inflation_kwargs = inflation_kwargs,
        year = year,
        data_year = data_year
        )
    df_sum = dataframe_by_group(survey_scenario, category = 'niveau_vie_decile', variables = simulated_variables, aggfunc = 'sum')
    agregates.loc[year] = df_sum.sum()

c:\Users\veve1\miniconda3\envs\openfiscaenv\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
c:\users\veve1\onedrive\documents\ensae 3a\memoire mie\openfisca-france-indirect-taxation\openfisca_france_indirect_taxation\projects\TVA_Herve_IPP\new_calage_bdf_cn.py:158: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data_bdf_postes_cn[poste] = 0


In [30]:
agregates['Conso BDF'] = agregates['depenses_tot'] + agregates['loyer_impute']
agregates.reset_index(inplace= True)
agregates.rename(columns = {'index' : 'year'}, inplace= True)
agregates

,year,tva_taux_plein,tva_taux_intermediaire,tva_taux_reduit,tva_taux_super_reduit,tva_total,depenses_tva_exonere,depenses_tva_taux_plein,depenses_tva_taux_intermediaire,depenses_tva_taux_reduit,depenses_tva_taux_super_reduit,rev_disponible,rev_disp_yc_loyerimpute,loyer_impute,depenses_tot,depenses_totales,pondmen,Conso BDF
0,2017,7.876044e+10,1.182956e+10,1.085693e+10,2.916588e+08,1.017386e+11,1.828582e+11,4.725627e+11,1.301252e+11,2.082556e+11,1.418017e+10,1.197964e+12,1.387077e+12,1.891127e+11,1.007982e+12,8.251237e+11,8.445859e+10,1.197095e+12
1,2018,8.112204e+10,1.241641e+10,1.108701e+10,2.906346e+08,1.049161e+11,1.907752e+11,4.867323e+11,1.365805e+11,2.126691e+11,1.413038e+10,1.234271e+12,1.426805e+12,1.925339e+11,1.040887e+12,8.501123e+11,8.445859e+10,1.233421e+12
2,2019,8.256655e+10,1.307885e+10,1.135869e+10,2.915240e+08,1.072956e+11,1.929483e+11,4.953993e+11,1.438674e+11,2.178804e+11,1.417362e+10,1.278664e+12,1.475381e+12,1.967170e+11,1.064269e+12,8.713206e+11,8.445859e+10,1.260986e+12
3,2020,7.678579e+10,9.625282e+09,1.092114e+10,2.828792e+08,9.761510e+10,1.906758e+11,4.607148e+11,1.058781e+11,2.094874e+11,1.375332e+10,1.289747e+12,1.489995e+12,2.002478e+11,9.805093e+11,7.898335e+11,8.445859e+10,1.180757e+12
4,2021,8.383946e+10,1.139929e+10,1.113713e+10,2.987876e+08,1.066747e+11,1.999309e+11,5.030368e+11,1.253922e+11,2.136304e+11,1.452677e+10,1.358404e+12,1.562187e+12,2.037834e+11,1.056517e+12,8.565861e+11,8.445859e+10,1.260300e+12
5,2022,8.862212e+10,1.542947e+10,1.227636e+10,3.059662e+08,1.166339e+11,2.136061e+11,5.317327e+11,1.697241e+11,2.354829e+11,1.487579e+10,1.434894e+12,1.642992e+12,2.080976e+11,1.165422e+12,9.518155e+11,8.445859e+10,1.373519e+12
6,2023,9.262292e+10,1.694231e+10,1.344158e+10,3.112216e+08,1.233180e+11,2.546692e+11,5.557376e+11,1.863654e+11,2.578338e+11,1.513130e+10,1.560302e+12,1.773999e+12,2.136968e+11,1.269737e+12,1.015068e+12,8.445859e+10,1.483434e+12
7,2024,9.528888e+10,1.742996e+10,1.382846e+10,3.201795e+08,1.268675e+11,2.619993e+11,5.717333e+11,1.917296e+11,2.652551e+11,1.556682e+10,1.605212e+12,1.825060e+12,2.198476e+11,1.306284e+12,1.044285e+12,8.445859e+10,1.526132e+12


In [31]:
cn_agregates_by_year = pd.DataFrame()
for year in range(2017,2024):
    cn_agregates = new_get_cn_aggregates(year)
    cn_agregates.drop(['poste_02_4', 'poste_04_4_3','rev_disp_yc_loyerimpute','rev_disponible'], axis= 0, inplace = True)
    cn_agregates_by_year = pd.concat([cn_agregates_by_year,pd.DataFrame(cn_agregates.sum())])
cn_agregates_by_year
cn_agregates_by_year.reset_index(inplace = True)
cn_agregates_by_year['year'] = cn_agregates_by_year['index'].apply(lambda x : x[-4:]).astype(int)
cn_agregates_by_year.drop('index', axis = 1, inplace= True)
cn_agregates_by_year.rename(columns = {0 : 'Conso CN'}, inplace= True)
cn_agregates_by_year

c:\Users\veve1\miniconda3\envs\openfiscaenv\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,Conso CN,year
0,1.197358e+12,2017
1,1.233693e+12,2018
2,1.261263e+12,2019
3,1.181017e+12,2020
4,1.260578e+12,2021
5,1.373821e+12,2022
6,1.483760e+12,2023


In [35]:
to_compare = agregates.loc[:,['Conso BDF']].merge(cn_agregates_by_year, left_index= True, right_index= True)
to_compare.set_index('year', inplace = True)
to_compare = to_compare* 1e-9 
to_compare = to_compare.round(1)

In [36]:
to_compare['Ecart BDF / CN'] = (to_compare['Conso CN'] / to_compare['Conso BDF'] -1)* 100
to_compare

,Conso BDF,Conso CN,Ecart BDF / CN
year,,,
2017,1197.1,1197.4,0.025061
2018,1233.4,1233.7,0.024323
2019,1261.0,1261.3,0.023791
2020,1180.8,1181.0,0.016938
2021,1260.3,1260.6,0.023804
2022,1373.5,1373.8,0.021842
2023,1483.4,1483.8,0.026965


In [9]:
to_compare['Growth_rate_CN'] = to_compare['Conso CN'].pct_change()
to_compare

,Conso BDF,Conso CN,year,Ecart BDF / CN,Growth_rate_CN
0,1.197095e+12,1.197358e+12,2017,0.022011,NaN
1,1.233421e+12,1.233693e+12,2018,0.022007,0.030346
2,1.260986e+12,1.261263e+12,2019,0.022001,0.022348
3,1.180757e+12,1.181017e+12,2020,0.022010,-0.063624
4,1.260300e+12,1.260578e+12,2021,0.022014,0.067366
5,1.373519e+12,1.373821e+12,2022,0.021994,0.089835
6,1.483434e+12,1.483760e+12,2023,0.021980,0.080024


**Par postes agrégés**

In [10]:
from openfisca_france_indirect_taxation.projects.TVA_Herve_IPP.new_calage_bdf_cn_by_postes_agreges import (get_bdf_aggregates,
    get_cn_aggregates,
    get_inflators_bdf_to_cn,
    get_inflators_cn_to_cn,
    get_inflators_by_year)

In [11]:
aggregates_13_postes = get_cn_aggregates(2023)
aggregates_13_postes


c:\Users\veve1\miniconda3\envs\openfiscaenv\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
c:\Users\veve1\miniconda3\envs\openfiscaenv\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,conso_CN_2023
Code,
poste_01,1.843824e+11
poste_02,5.122070e+10
poste_03,4.420050e+10
poste_04,4.043368e+11
poste_05,5.897990e+10
poste_06,5.615550e+10
poste_07,1.917514e+11
poste_08,5.294940e+10
poste_09,1.026023e+11


In [12]:
cn_agregates = new_get_cn_aggregates(2023)
to_substract = cn_agregates.loc[cn_agregates.index.isin(['poste_02_4', 'poste_04_4_3','loyer_impute'])]
to_substract.rename(index ={'loyer_impute' : 'poste_04_2'}, columns = {'conso_CN_2023' : 'to_substract'}, inplace = True)
to_substract = to_substract.groupby(to_substract.index.str[:8]).sum()

c:\Users\veve1\miniconda3\envs\openfiscaenv\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [13]:
aggregates_13_postes = aggregates_13_postes.merge(to_substract, left_index= True, right_index= True, how = 'left')
aggregates_13_postes.fillna(0, inplace = True)

In [14]:
aggregates_13_postes['right_conso_CN_2023'] = aggregates_13_postes['conso_CN_2023'] - aggregates_13_postes['to_substract']
aggregates_13_postes.drop(columns=['conso_CN_2023','to_substract'], axis= 1,  inplace= True)
aggregates_13_postes.rename(columns={'right_conso_CN_2023' : 'conso_CN_2023'}, inplace = True)

aggregates_13_postes.index = aggregates_13_postes.index.to_series().apply(lambda x :x.replace('poste_' ,'poste_agrege_'))

In [15]:
data_year = 2017
inflators = new_get_inflators(2023,2017)
simulated_variables = ["poste_agrege_0{}".format(i) for i in range(1, 10)] + ["poste_agrege_10", "poste_agrege_11", "poste_agrege_12", "poste_agrege_13"]

postes_agreges = pd.DataFrame(columns=["poste_agrege_0{}".format(i) for i in range(1, 10)] + ["poste_agrege_10", "poste_agrege_11", "poste_agrege_12", "poste_agrege_13"])
inflation_kwargs = dict(inflator_by_variable = inflators)
survey_scenario = SurveyScenario.create(
    inflation_kwargs = inflation_kwargs,
    year = 2023,
    data_year = data_year
    )
df_sum = dataframe_by_group(survey_scenario, category = 'niveau_vie_decile', variables = simulated_variables, aggfunc = 'sum')
postes_agreges = df_sum.sum()
conso_by_poste = pd.DataFrame(postes_agreges.T).reset_index().rename(columns={'index' : 'Code', 0 : 'conso_OFF_IT_2023'}).set_index('Code')

c:\users\veve1\onedrive\documents\ensae 3a\memoire mie\openfisca-france-indirect-taxation\openfisca_france_indirect_taxation\projects\TVA_Herve_IPP\new_calage_bdf_cn.py:158: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data_bdf_postes_cn[poste] = 0


In [ ]:
aggregates_13_postes = aggregates_13_postes.merge(conso_by_poste, left_index= True, right_index= True)
aggregates_13_postes = aggregates_13_postes * 1e-9
aggregates_13_postes = aggregates_13_postes.round(2)
aggregates_13_postes.rename({'poste_agrege_13' : 'poste_agrege_12' },axis = 0, inplace = True)
aggregates_13_postes = aggregates_13_postes.groupby(aggregates_13_postes.index).sum()

In [55]:
label_postes_agreges = ['Alimentation', 'Alcools et tabacs', 'Habillement et chaussures', 'Logement', 'Ameublement et équipement ménager', 'Santé',
                        'Transports', 'Communication' , 'Loisirs et cultures' , 'Education', 'Restauration et hôtellerie', 'Services et biens divers']
liste_postes_agreges = ["poste_agrege_0{}".format(i) for i in range(1, 10)] + ["poste_agrege_{}".format(i) for i in range(10, 13)]
label_dict = dict(zip(liste_postes_agreges,label_postes_agreges))

In [59]:
aggregates_13_postes.rename(label_dict, axis = 0, inplace = True)

In [60]:
aggregates_13_postes

,conso_CN_2023,conso_OFF_IT_2023
Code,,
Alimentation,184.38,184.34
Alcools et tabacs,46.13,46.12
Habillement et chaussures,44.20,44.19
Logement,183.76,183.72
Ameublement et équipement ménager,58.98,58.97
Santé,56.16,56.14
Transports,191.75,191.71
Communication,52.95,52.94
Loisirs et cultures,102.60,102.58
